# Automatic Land Cover Classification



![Dataset sample](https://i.ibb.co/ZYfpM52/bak-sat.png)

*Helber, P., Bischke, B., Dengel, A., & Borth, D. (2018). EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification [Data set]. In EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification (Vol. 12, No. 7, pp. 2217-2226). Zenodo. https://doi.org/10.5281/zenodo.7711810*

In this exercise, we will use an open dataset for land use classification. This dataset is derived from the Sentinel-2 satellite and is available in both RGB format (which we will use) and TIFF format, with the satellite's 13 bands, providing much more information but making processing computationally more complex.

To analyze, process, or extract any added value from remote sensing data, it is necessary to choose a working environment and have data availability. For this, we have different options.

### Working Environments
We can choose Geographic Information Systems, tools from various agencies such as SNAP, services like Google Earth Engine (GEE), or use our own code. More and more examples are available in *Jupyter notebooks*, which is what we will use. To run them, we can install Jupyter on our PC or use a cloud service, such as:

* Jupyterhub Sentinel Hub: https://jupyterhub.dataspace.copernicus.eu/ (registration required)
* Google Colab

### Data
Depending on who generates the data and the available instruments, sometimes we need to access data differently. A very flexible service is GEE, which offers data from various sources. However, if we learn to select and download any type of data, we will have much more flexibility. In the second part of the exercise, we will focus on downloading Sentinel-2 data using the *Sentinel Hub Process API* service.

We will see how to use some configuration parameters to obtain preprocessed or raw products. For more information, you can check the [official documentation](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html).

### Imports
Programming languages come with libraries, which are sets of functions that facilitate performing various tasks, such as loading images or designing neural networks. Among the ones we will use are TensorFlow (for training neural networks and other AI techniques), matplotlib (for plots), and numpy (for data management), among others.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import random

In the following block, we download the available data in RGB format and extract it. If you look at the generated folder, it is divided into different folders according to the type of land cover (10 types or classes).

In [ ]:
# Descargar el dataset EuroSAT
!wget --no-check-certificate -O EuroSAT.zip https://madm.dfki.de/files/sentinel/EuroSAT.zip

# Descomprimir el archivo
with zipfile.ZipFile('EuroSAT.zip', 'r') as zip_ref:
    zip_ref.extractall('EuroSAT')

# Definir el directorio del dataset
dataset_dir = 'EuroSAT/2750'

In the following block, we prepare "data generators" that read the images we have downloaded and prepare them for training the AI model.

We will split the data into training (80%) and validation (20%) sets. This will help us check during each epoch (or weight calculation iteration) whether the network we are training is overfitting or not. "Overfitting" occurs when the network learns very well to differentiate classes within the training data but does not perform as well with new data. What we want is to avoid this so we can use our network for other images or generalize.

In [ ]:
# Crear generadores de datos para entrenamiento y validación
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Now it's time to configure our network. One of the things we can do is **transfer learning**, which involves starting from a pre-existing generic model (e.g., for image classification) and fine-tuning it for our specific case of satellite images.

For practical purposes, we will build a network from scratch by adding different layers. Since we are working with image data, convolutional neural networks usually perform well, so we will add the following layers:

**Conv2D Layer**

- **Type:** Convolutional  
- **Description:** This is the first convolutional layer, which applies 32 filters (or kernels) of size 3x3 over the input image.  
- **Function:** Detects local features in the image, such as edges, textures, etc.

**MaxPooling2D Layer**

- **Description:** Reduces the dimensionality of the input (image) by taking the maximum value from each 2x2 block.  
- **Function:** Decreases the spatial resolution of detected features, reducing the number of parameters and computational load.

**Dropout Layer**

- **Description:** During training, this layer randomly "turns off" 25% of the units (neurons) to prevent overfitting.  
- **Function:** Helps prevent the model from fitting too closely to the training data, improving its generalization ability.

**Flatten Layer**

- **Description:** Converts the 2D (matrix) output into a 1D (vector) form so it can be fed into densely connected layers.  
- **Function:** Prepares data for the dense (fully connected) layers.

**Dense Layer**

- **Function:** Processes the extracted features and learns non-linear combinations of these features.

**Dense(13, activation='softmax') Layer**

- **Description:** Output layer with 13 units, one for each class in the EuroSAT dataset.  
- **Function:** Produces a probability distribution over the 13 classes.  
- **Activation:** Softmax, which ensures that the sum of the outputs is 1, representing a probability for each class.

Feel free to try adding or removing layers and observe what happens.


In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models

# Definir la arquitectura del modelo
num_classes = len(train_generator.class_indices)  # Get the actual number of classes

model = models.Sequential([
    layers.Conv2D(???filters???, (3, 3), activation='relu', input_shape=??image shape and bands),
    ....
])

# Compilar el modelo
model.compile(...)


It's time for training. The model is divided into different **epochs**, where the data passes through the network and certain weights are calculated to minimize a loss function.

For each epoch, 80% of the data is used for training, and 20% is used to check if the model performs well on data it hasn't seen before.


In [ ]:
# Entrenar el modelo con el callback de TensorBoard
history = model.fit(...)

# Evaluar el modelo
loss, accuracy = model.evaluate(validation_generator, verbose=2)
print(f'Validation accuracy: {accuracy*100:.2f}%')

model.save('eurosat_model.keras')

Once the training is complete, we can check for overfitting by displaying various graphs.

If there is a significant difference between the accuracy and the loss function for the training and validation data—where the validation data performs worse—there may be overfitting.

This can be addressed in different ways, such as "disconnecting" parts of the network layers using **Dropout**.


In [ ]:
# Visualizar los resultados
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model evaluation
In addition to the **accuracy** that the model provides, one of the key plots used to evaluate AI models is the **confusion matrix**.

For a test dataset, we will use the network to predict the class of each sample, knowing its true class. The matrix shows the relationship between the actual and predicted values, displaying the number of instances for each case (actual vs. predicted).

For a well-performing model, it is expected that the diagonal of the matrix is clearly highlighted, indicating correct predictions.


In [ ]:
# Obtener todas las imágenes y etiquetas del conjunto de validación
# Iterar sobre el generador para obtener todas las muestras
validation_images = []
validation_labels = []
for _ in range(len(validation_generator)):
    batch_images, batch_labels = next(validation_generator)
    validation_images.append(batch_images)
    validation_labels.append(batch_labels)

# Concatenar los lotes en un solo array
validation_images = np.concatenate(validation_images, axis=0)
validation_labels = np.concatenate(validation_labels, axis=0)

# Seleccionar aleatoriamente un subconjunto de las imágenes de validación
num_samples = 1000  # Número de muestras aleatorias para la matriz de confusión
# Asegurarse de que num_samples no sea mayor que el número de imágenes disponibles
num_samples = min(num_samples, len(validation_images))
indices = np.random.choice(len(validation_images), num_samples, replace=False)
random_images = validation_images[indices]
random_labels = validation_labels[indices]

# Hacer predicciones sobre estas imágenes aleatorias
predictions = model.predict(random_images)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(random_labels, axis=1)

# Generar la matriz de confusión
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=validation_generator.class_indices.keys(), yticklabels=validation_generator.class_indices.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

We can also visually evaluate the model by randomly displaying an image to compare the actual and predicted class.

This allows us to better understand how well the model performs with real data and to identify potential misclassifications.


In [ ]:
# Imprimir el reporte de clasificación
print(classification_report(true_labels, predicted_labels, target_names=validation_generator.class_indices.keys()))

# Mostrar algunas imágenes con sus etiquetas verdaderas y predichas
plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(random_images[i])
    true_label = list(validation_generator.class_indices.keys())[true_labels[i]]
    predicted_label = list(validation_generator.class_indices.keys())[predicted_labels[i]]
    plt.title(f'True: {true_label}\nPred: {predicted_label}')
    plt.axis('off')
plt.show()


## Model Usage: Data Download

Now let's use the model to make some predictions. In this case, we will download an image from **Sentinel Hub** and test different parts of it.

For this, we need the **SentinelHub** library:


In [ ]:
!pip install sentinelhub
from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

### Credentials

If you haven't done so yet, the first step is to register on the [Copernicus Data Space](https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=sh-a696e3be-b074-4baa-9e76-b10bee279c85&tab_id=Ns0w8gGsZac).

The credentials for Sentinel Hub services (`client_id` and `client_secret`) can be obtained from your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the User Settings, you can create a new OAuth Client to generate these credentials. For more detailed instructions, visit the relevant [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Now that you have your `client_id` and `client_secret`, it is recommended to set up a new profile in your **Sentinel Hub Python package**. Instructions on how to configure your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). By following these instructions, you can create a specific profile to use the package for accessing the data collections from the **Copernicus Data Space Ecosystem**. This is useful because changes in the configuration class are usually temporary in a notebook, and by saving the configuration in your profile, you won’t need to generate new credentials or overwrite/change the default profile each time you run or write a new Jupyter Notebook.

If this is your first time using the Sentinel Hub Python package for the **Copernicus Data Space Ecosystem**, you must create a specific profile for it. You can do this in the following cell:


In [ ]:
config = SHConfig()
config.sh_client_id = 'sh-7b087513-8281-4caf-9120-f26c486a06e2'
config.sh_client_secret = 'isk7TcXlC1U4iScfSCLcqFnAa0Y4qr4d'
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.sh_auth_base_url="https://services.sentinel-hub.com",
config.save("uimp_24")

## Choosing Area of Interest

We are going to download a Sentinel-2 image of the [Bay of Santander](https://en.wikipedia.org/wiki/Bay_of_Santander) like the one shown below (captured by Sentinel-2 on 2024-04-20):  
![santander.png](https://i.ibb.co/bF6mtYW/sdr-sat.png)


The selected box in `WGS84` coordinates is `[-3.855858, 43.395070, -3.721447, 43.499383]` (longitude and latitude coordinates from the bottom-left to the top-right).

If you want to select another area, you can use tools like [bboxfinder](http://bboxfinder.com/).

All server requests require an area represented by a `sentinelhub.geometry.BBox` instance with the corresponding reference system (`sentinelhub.constants.CRS`).

Once the bounding box is defined, you can initialize `BBox` with the area of interest. Using the `bbox_to_dimensions` function, we can calculate the exact resolution of the image.


In [ ]:
bahia_coords_wgs84 = (-3.855858, 43.395070, -3.721447, 43.499383)
resolution = 10
bahia_bbox = BBox(bbox=bahia_coords_wgs84, crs=CRS.WGS84)
bahia_size = bbox_to_dimensions(bahia_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {bahia_size} pixels")

In the documentation, you can find many configuration parameters for downloading data.

In this case, we will request the **Red**, **Green**, and **Blue (RGB)** bands, which are the ones we used to train our model.

However, we can also train and use the model with all available bands.


In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2024-04-19", "2024-04-21"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.JPG)],
    bbox=bahia_bbox,
    size=bahia_size,
    config=config,
)

In [ ]:
true_color_imgs = request_true_color.get_data()

Let's display the downloaded image using a helper function called `plot_image`.

In [ ]:
"""
Utility functions used by example notebooks
"""
from typing import Any, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np


def plot_image(
    image: np.ndarray,
    factor: float = 1.0,
    clip_range: Optional[Tuple[float, float]] = None,
    **kwargs: Any
) -> None:
    """Utility function for plotting RGB images."""
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

image = true_color_imgs[0]
print(f"Image type: {image.dtype}")

# plot function
# factor 1/255 to scale between 0-1
# factor 3.5 to increase brightness
plot_image(image, factor=3.0 / 255, clip_range=(0, 1))

The following code takes the downloaded image and selects random 64x64 patches, which is the resolution we used to train the model.

In [ ]:
!pip install Pillow
from PIL import Image

def load_and_split_image(image_array, target_size=(64, 64), num_crops=9):
    # Convert NumPy array to PIL Image
    image = Image.fromarray(image_array.astype(np.uint8))  # Assuming image_array is uint8 type

    height, width = image.size  # Use PIL's size attribute
    crops = []
    for _ in range(num_crops):
        # Seleccionar aleatoriamente la esquina superior izquierda del recorte
        left = np.random.randint(0, width - target_size[0])
        top = np.random.randint(0, height - target_size[1])
        right = left + target_size[0]
        bottom = top + target_size[1]

        # Recortar la imagen y convertirla a un array de numpy
        crop = image.crop((left, top, right, bottom))
        crop_array = np.array(crop) * 3.0 / 255.0  # Normalizar los valores de los píxeles
        crops.append(crop_array)

    return np.array(crops)

# Assuming 'image' is the NumPy array you want to process
crops = load_and_split_image(image)

We predict the categories of the different patches and display them with the predicted label.

In [ ]:
# Hacer predicciones sobre los trozos
predictions = model.predict(crops)

# Obtener las etiquetas predichas para cada trozo
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
from scipy.stats import mode

# Encontrar la etiqueta más frecuente entre las predicciones
final_prediction = mode(predicted_labels).mode

# Obtener el nombre de la clase predicha
predicted_class = list(validation_generator.class_indices.keys())[final_prediction] # The variable final_prediction is now a scalar and does not need to be indexed.

print(f'Predicted class for the image: {predicted_class}')

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(len(crops)):
    plt.subplot(3, 3, i+1)  # Crear una cuadrícula de 3x3
    plt.imshow(crops[i])
    predicted_class = list(validation_generator.class_indices.keys())[predicted_labels[i]]
    plt.title(f'Pred: {predicted_class}')
    plt.axis('off')
plt.show()